# 1. Introduction
## 1.1 A voice transformation tool,
that can listen to your voice, change it to another language while maintaining your tone, genre, and slang.

## 1.2 Here are some steps to follow:

1. **Voice Recording:** Use a voice recording app to capture your voice.
2. **Translation:** Use a translation tool like *Google Translate or DeepL* to translate your text into the desired language.
3. **Voice Synthesis:** Use a *text-to-speech (TTS)* tool that supports the target language and can mimic different tones and genres. Some advanced **TTS** tools include
  * Amazon Polly,
  * Google Text-to-Speech, and
  * Microsoft Azure TTS.
4. **Voice Modulation:** If you need to further adjust the tone and slang, you might need a *voice modulation software* like **Voicemod** or **MorphVOX**.

basic implementation for
* transforming voice recordings,
* translating them, and then
* synthesizing the translated text back into speech.

# 2. Record voice
use the `pyaudio` library to record voice.

In [3]:
!sudo apt install portaudio19-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (209 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEn

In [16]:
# !sudo apt-get install python-pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python-pyaudio


In [4]:
# load the 'pyaudio' library
# !pip -v install pyaudio
!pip install pyaudio

# !pip3 -v install PyAudio

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=PyAudio-0.2.14-cp310-cp310-linux_x86_64.whl size=63856 sha256=a7fb15e151b6753b343787fbb808943d5c4473cba7c4ec20c306a78a89a4c31a
  Stored in directory: /root/.cache/pip/wheels/d6/21/f4/0b51d41ba79e51b16295cbb096ec49f334792814d545b508c5
Successfully built pyaudio


In [5]:
import pyaudio
import wave

In [6]:
p = pyaudio.PyAudio()
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')

print('hello numdevices', numdevices)

for i in range(0, numdevices):
        if (p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", p.get_device_info_by_host_api_device_index(0, i).get('name'))

p.terminate()

hello numdevices 0


In [15]:
# Import packages
import base64
import google.colab
import IPython.display

# Define record in js
record_js = """  # Changed variable name to record_js to avoid conflict with the function name
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(duration=3):
  print(f"{duration}s recording in progress")
  display(IPython.display.Javascript(record_js)) # Pass the JavaScript code string
  duration += 1
  s = google.colab.output.eval_js('record_js(%d)' % (duration*1000))
  print("Audio recording complete")
  b = base64.b64decode(s.split(',')[1])
  return b

test = IPython.display.Audio(record())
IPython.display.display(test)

3s recording in progress


<IPython.core.display.Javascript object>

MessageError: ReferenceError: record_js is not defined

In [7]:
def record_audio(filename, duration=5):
    chunk = 1024
    sample_format = pyaudio.paInt16
    channels = 1
    fs = 44100
    p = pyaudio.PyAudio()

    print('Recording...')
    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)
                    # input_device_index=input_device_index) # Add this line with the correct device index
    frames = []

    for _ in range(0, int(fs / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    print('Finished recording.')

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()

record_audio('output.wav')

Recording...


OSError: [Errno -9996] Invalid input device (no default output device)